In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings
import joblib
import os
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',None)

In [2]:
file_list = []
for root_dir,sub_dir,files in os.walk('./data'):
    file_list.append(files)
file_list = file_list[0]
dataframe_list = pd.DataFrame()
for file in files:
    if file.endswith('xls'):
        file_name = os.path.join(root_dir,file)
        df = pd.read_excel(file_name)
        dataframe_list = dataframe_list.append(df)

In [3]:
dataframe_list.drop(['Unnamed: 18','Unnamed: 19','Unnamed: 20','Unnamed: 21','Unnamed: 22','Unnamed: 23','创角账号数'],
                    axis=1,inplace=True)
dataframe_list.columns = ['plan_name','channel_id','source_id','data_win','source_run_datetime','create_time','plan_id',
                         'game_id','platform','source_run_date_amount','create_role_num','create_role_cost','create_role_pay_num',
                          'create_role_pay_cost','create_role_pay_sum','create_role_roi','create_role_retain_1d']
dataframe_list.dropna(how='any',inplace=True)

In [4]:
dataframe_list.shape

(1718817, 17)

In [5]:
dataframe_list = dataframe_list[dataframe_list['source_run_date_amount'] > 0]

In [6]:
dataframe_list.shape

(399792, 17)

In [7]:
dataframe_list.head()

,plan_name,channel_id,source_id,data_win,source_run_datetime,create_time,plan_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_retain_1d
1,番茄小说-末日安卓,7162.0,299591.0,0.0,2020-05-01,2020-04-24,289813.0,1001049.0,1.0,200000.00,690.0,290.275762,3.0,66666.666667,318.0,0.001590,0.0522
3,mei207-安卓末生-5.1_3,20754.0,304584.0,0.0,2020-05-01,2020-05-01,294806.0,1001049.0,1.0,6603.77,640.0,10.334538,3.0,2201.256667,66.0,0.009994,0.0516
4,t5i-165-mrwc-4.29n_6,20754.0,304143.0,0.0,2020-05-01,2020-04-30,294365.0,1000840.0,1.0,6640.08,406.0,16.476625,2.0,3320.040000,218.0,0.032831,0.0571
5,s0103-mrwc-4.27_1,20754.0,301128.0,0.0,2020-05-01,2020-04-27,291350.0,1000840.0,1.0,3850.09,270.0,14.474023,3.0,1283.363333,66.0,0.017143,0.0451
6,7tt_681_末世生机_2,6770.0,301064.0,0.0,2020-05-01,2020-04-27,291286.0,1001049.0,1.0,1621.59,256.0,6.409447,0.0,0.000000,0.0,0.000000,0.0593


In [8]:
dataframe_list['data_win'].value_counts()

7.0    80414
6.0    71631
5.0    62748
4.0    53757
3.0    44636
2.0    35315
0.0    25724
1.0    25567
Name: data_win, dtype: int64

In [45]:
dataframe_list.sort_values(by='data_win',inplace=True)

In [46]:
dataframe_list.drop_duplicates(subset=['channel_id','source_id','source_run_date_amount','create_role_num'],keep='first'
                               ,inplace=True)

In [47]:
dataframe_list.shape

(102843, 17)

In [48]:
dataframe_list['channel_id'] = dataframe_list['channel_id'].astype(np.int64)
dataframe_list['source_id'] = dataframe_list['source_id'].astype(np.int64)
dataframe_list['data_win'] = dataframe_list['data_win'].astype(np.int64)
dataframe_list['plan_id'] = dataframe_list['plan_id'].astype(np.int64)
dataframe_list['platform'] = dataframe_list['platform'].astype(np.int64)
dataframe_list['game_id'] = dataframe_list['game_id'].astype(np.int64)
dataframe_list['source_run_datetime'] = pd.to_datetime(dataframe_list['source_run_datetime'])

In [49]:
dataframe_list.head()

,plan_name,channel_id,source_id,data_win,source_run_datetime,create_time,plan_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_retain_1d
1,番茄小说-末日安卓,7162,299591,0,2020-05-01,2020-04-24,289813,1001049,1,200000.00,690.0,290.275762,3.0,66666.666667,318.0,0.00159,0.0522
197,HSX-TT-WQ-废土-0518_4,20541,311564,0,2020-05-19,2020-05-18,301786,1000993,2,657.55,3.0,219.183333,0.0,0.000000,0.0,0.00000,0.0000
198,HSX-TT-WQ-废土-0518_6,20541,311566,0,2020-05-19,2020-05-18,301788,1000993,2,384.88,3.0,128.293333,0.0,0.000000,0.0,0.00000,0.0000
199,mei-末日-new-5.18_2,20953,311781,0,2020-05-19,2020-05-18,302003,1001049,1,274.87,3.0,91.623333,0.0,0.000000,0.0,0.00000,0.0000
200,zhu-废土-IOS-5.18_2,20954,311786,0,2020-05-19,2020-05-18,302008,1000993,2,274.48,3.0,91.493333,0.0,0.000000,0.0,0.00000,0.0000


In [50]:
# 读取头条5-7月份计划30日回款信息
pay_info = pd.read_csv('./30_d_pay_sum.csv')
pay_info['source_run_datetime'] = pd.to_datetime(pay_info['source_run_datetime'])
pay_info.drop(pay_info[pay_info['channel_id']==-1].index,inplace=True)

In [51]:
data_pay_temp = dataframe_list[dataframe_list['data_win'] == 0]

In [52]:
data_pay_temp.shape

(25708, 17)

In [53]:
data_pay_temp.head()

,plan_name,channel_id,source_id,data_win,source_run_datetime,create_time,plan_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_retain_1d
1,番茄小说-末日安卓,7162,299591,0,2020-05-01,2020-04-24,289813,1001049,1,200000.00,690.0,290.275762,3.0,66666.666667,318.0,0.00159,0.0522
197,HSX-TT-WQ-废土-0518_4,20541,311564,0,2020-05-19,2020-05-18,301786,1000993,2,657.55,3.0,219.183333,0.0,0.000000,0.0,0.00000,0.0000
198,HSX-TT-WQ-废土-0518_6,20541,311566,0,2020-05-19,2020-05-18,301788,1000993,2,384.88,3.0,128.293333,0.0,0.000000,0.0,0.00000,0.0000
199,mei-末日-new-5.18_2,20953,311781,0,2020-05-19,2020-05-18,302003,1001049,1,274.87,3.0,91.623333,0.0,0.000000,0.0,0.00000,0.0000
200,zhu-废土-IOS-5.18_2,20954,311786,0,2020-05-19,2020-05-18,302008,1000993,2,274.48,3.0,91.493333,0.0,0.000000,0.0,0.00000,0.0000


In [63]:
data_pay = pd.merge(data_pay_temp,pay_info,on=['channel_id','source_id','source_run_datetime'],how='left',
                validate='one_to_one')

In [70]:
print('总消耗',data_pay['source_run_date_amount'].sum())
print('有消耗没有创角的计划消耗',data_pay[data_pay['30_d_pay_sum'].isna()]['source_run_date_amount'].sum())
print('占比',data_pay[data_pay['30_d_pay_sum'].isna()]['source_run_date_amount'].sum()/data_pay['source_run_date_amount'].sum())

总消耗 31833991.77
有消耗没有创角的计划消耗 626641.91
占比 0.019684679022583036


In [71]:
data_pay.dropna(how='any',inplace=True)

In [72]:
data_pay.shape

(20694, 18)

In [73]:
data_pay['30_ROI'] = round(data_pay['30_d_pay_sum']/data_pay['source_run_date_amount'],6)

In [74]:
data_pay = data_pay[['channel_id','source_id','source_run_datetime','30_d_pay_sum','30_ROI']]

In [75]:
data_pay.head()

,channel_id,source_id,source_run_datetime,30_d_pay_sum,30_ROI
0,7162,299591,2020-05-01,4654.0,0.02327
1,20541,311564,2020-05-19,0.0,0.00000
2,20541,311566,2020-05-19,0.0,0.00000
3,20953,311781,2020-05-19,0.0,0.00000
4,20954,311786,2020-05-19,0.0,0.00000


In [76]:
data = pd.merge(dataframe_list,data_pay,on=['channel_id','source_id','source_run_datetime'],how='left',
                validate='many_to_one')
data.dropna(how='any',inplace=True)
data.head()

,plan_name,channel_id,source_id,data_win,source_run_datetime,create_time,plan_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_retain_1d,30_d_pay_sum,30_ROI
0,番茄小说-末日安卓,7162,299591,0,2020-05-01,2020-04-24,289813,1001049,1,200000.00,690.0,290.275762,3.0,66666.666667,318.0,0.00159,0.0522,4654.0,0.02327
1,HSX-TT-WQ-废土-0518_4,20541,311564,0,2020-05-19,2020-05-18,301786,1000993,2,657.55,3.0,219.183333,0.0,0.000000,0.0,0.00000,0.0000,0.0,0.00000
2,HSX-TT-WQ-废土-0518_6,20541,311566,0,2020-05-19,2020-05-18,301788,1000993,2,384.88,3.0,128.293333,0.0,0.000000,0.0,0.00000,0.0000,0.0,0.00000
3,mei-末日-new-5.18_2,20953,311781,0,2020-05-19,2020-05-18,302003,1001049,1,274.87,3.0,91.623333,0.0,0.000000,0.0,0.00000,0.0000,0.0,0.00000
4,zhu-废土-IOS-5.18_2,20954,311786,0,2020-05-19,2020-05-18,302008,1000993,2,274.48,3.0,91.493333,0.0,0.000000,0.0,0.00000,0.0000,0.0,0.00000


In [512]:
data.shape

(44703, 19)

In [513]:
data_ignore = data[data['source_run_date_amount'] < 500]
data_not_ignore = data[data['source_run_date_amount'] >= 500]

In [514]:
data_an = data_not_ignore[data_not_ignore['platform'] == 1]
data_ios = data_not_ignore[data_not_ignore['platform'] == 2]

In [515]:
data_win_0 = data_an[data_an['data_win'] == 0]
data_win_1 = data_an[data_an['data_win'] == 1]
data_win_2 = data_an[data_an['data_win'] == 2]
data_win_3 = data_an[data_an['data_win'] == 3]
data_win_4 = data_an[data_an['data_win'] == 4]
data_win_5 = data_an[data_an['data_win'] == 5]
data_win_6 = data_an[data_an['data_win'] == 6]
data_win_7 = data_an[data_an['data_win'] == 7]
data_win_0_ios = data_ios[data_ios['data_win'] == 0]
data_win_1_ios = data_ios[data_ios['data_win'] == 1]
data_win_2_ios = data_ios[data_ios['data_win'] == 2]
data_win_3_ios = data_ios[data_ios['data_win'] == 3]
data_win_4_ios = data_ios[data_ios['data_win'] == 4]
data_win_5_ios = data_ios[data_ios['data_win'] == 5]
data_win_6_ios = data_ios[data_ios['data_win'] == 6]
data_win_7_ios = data_ios[data_ios['data_win'] == 7]

In [516]:
data_win_0.head()

,plan_name,channel_id,source_id,data_win,source_run_datetime,create_time,plan_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_retain_1d,30_d_pay_sum,30_ROI
0,番茄小说-末日安卓,7162,299591,0,2020-05-01,2020-04-24,289813,1001049,1,200000.00,690.0,290.275762,3.0,66666.666667,318.0,0.00159,0.0522,4654.0,0.023270
7,全民投资4-末日-new-5.19_2,20953,312091,0,2020-05-19,2020-05-19,302313,1000840,1,687.12,3.0,229.040000,0.0,0.000000,0.0,0.00000,0.3333,12.0,0.017464
9,ba-末日-new-5.19_1,20953,312103,0,2020-05-19,2020-05-19,302325,1000840,1,552.79,3.0,184.263333,0.0,0.000000,0.0,0.00000,0.0000,0.0,0.000000
10,ku11-mrwc双-5.19_1,20754,312205,0,2020-05-19,2020-05-19,302427,1000840,1,596.07,3.0,198.690000,0.0,0.000000,0.0,0.00000,0.3333,0.0,0.000000
11,t5i-165-mrwc-5.19_7,20754,312325,0,2020-05-19,2020-05-19,302547,1000840,1,865.32,3.0,288.440000,0.0,0.000000,0.0,0.00000,0.0000,0.0,0.000000


In [89]:
# data_win_0['label'] = data_win_0.apply(lambda x:1 if x.create_role_num == 0 else 
#                                 (1 if x.create_role_cost>=100 else (1 if x.create_role_pay_sum==0 else 2) ),axis=1)
# data_win_0_ios['label'] = data_win_0_ios.apply(lambda x:1 if x.create_role_num == 0 else 
#                                 (1 if x.create_role_cost>=200 else (1 if x.create_role_pay_sum==0 else 2) ),axis=1)

In [520]:
data_win_0['label'] = data_win_0.apply(lambda x:1 if x.create_role_num == 0 else (1 if (x.create_role_cost>=20)&
        (x.create_role_pay_num==0) else(0 if (x.create_role_cost<=70)&(x.create_role_roi>=0.02) else 
                                        (1 if x.create_role_cost>=50 else 2))),axis=1)
data_win_0_ios['label'] = data_win_0_ios.apply(lambda x:1 if x.create_role_num == 0 else (1 if (x.create_role_cost>=40)&
        (x.create_role_pay_num==0) else(0 if (x.create_role_cost<=300)&(x.create_role_roi>=0.02) else 
                                        (1 if x.create_role_cost>=120 else 2))),axis=1)
# data_win_0_ios['label'] = data_win_0_ios.apply(lambda x:1 if x.create_role_num == 0 else 
#                                 (1 if x.create_role_cost>=120 else 2),axis=1)

In [303]:
# data_win_0['label'] = data_win_0.apply(lambda x:1 if x.create_role_num == 0 else 
#                                 (1 if x.create_role_cost>=50 else 2),axis=1)
# data_win_0_ios['label'] = data_win_0_ios.apply(lambda x:1 if x.create_role_num == 0 else 
#                                 (1 if x.create_role_cost>=120 else 2),axis=1)

In [521]:
data_win_0['label'].value_counts()

1    5414
2    2265
0     919
Name: label, dtype: int64

In [522]:
data_win_0_ios['label'].value_counts()

1    2518
2     808
0     328
Name: label, dtype: int64

In [523]:
result_df = pd.DataFrame()
temp_df = pd.DataFrame()

In [524]:
result_df = result_df.append(data_win_0[(data_win_0['label'] == 1)| (data_win_0['label'] == 0)]).append(data_win_0_ios[(data_win_0_ios['label'] == 1)|(data_win_0_ios['label'] == 0)])
temp_df = temp_df.append(data_win_0[data_win_0['label'] == 2]).append(data_win_0_ios[data_win_0_ios['label'] == 2])

In [300]:
# # win = 1
# data_win_1['label'] = data_win_1.apply(lambda x:1 if x.create_role_num == 0 else 
#                                 (1 if x.create_role_cost>=50 else 2),axis=1)
# data_win_1_ios['label'] = data_win_1_ios.apply(lambda x:1 if x.create_role_num == 0 else 
#                                 (1 if x.create_role_cost>=120 else 2),axis=1)
# result_df = result_df.append(data_win_1[data_win_1['label'] == 1]).append(data_win_1_ios[data_win_1_ios['label'] == 1])
# temp_df = temp_df.append(data_win_1[data_win_1['label'] == 2]).append(data_win_1_ios[data_win_1_ios['label'] == 2])

In [525]:
# win = 1
data_win_1['label'] = data_win_1.apply(lambda x:1 if x.create_role_num == 0 else (0 if (x.create_role_cost<=70)&(x.create_role_roi>=0.02)
                                else (1 if x.create_role_cost>=50 else (1 if x.create_role_pay_sum==0 else 2))),axis=1)
data_win_1_ios['label'] = data_win_1_ios.apply(lambda x:1 if x.create_role_num == 0 else (0 if (x.create_role_cost<=300)&(x.create_role_roi>=0.02)
                                else (1 if x.create_role_cost>=120 else (1 if x.create_role_pay_sum==0 else 2))),axis=1)
result_df = result_df.append(data_win_1[(data_win_1['label'] == 1)| (data_win_1['label'] == 0)]).append(data_win_1_ios[(data_win_1_ios['label'] == 1)|(data_win_1_ios['label'] == 0)])
temp_df = temp_df.append(data_win_1[data_win_1['label'] == 2]).append(data_win_1_ios[data_win_1_ios['label'] == 2])

In [144]:
# # win = 2
# data_win_2['label'] = data_win_2.apply(lambda x:1 if x.create_role_num == 0 else 
#                                 (1 if x.create_role_cost>=50 else (1 if x.create_role_pay_sum==0 else 2)),axis=1)
# data_win_2_ios['label'] = data_win_2_ios.apply(lambda x:1 if x.create_role_num == 0 else 
#                                 (1 if x.create_role_cost>=120 else (1 if x.create_role_pay_sum==0 else 2)),axis=1)
# result_df = result_df.append(data_win_2[data_win_2['label'] == 1]).append(data_win_2_ios[data_win_2_ios['label'] == 1])
# temp_df = temp_df.append(data_win_2[data_win_2['label'] == 2]).append(data_win_2_ios[data_win_2_ios['label'] == 2])

In [526]:
# win = 2
data_win_2['label'] = data_win_2.apply(lambda x:1 if x.create_role_num == 0 else (0 if (x.create_role_cost<=70)&(x.create_role_roi>=0.045)
                                else (1 if x.create_role_cost>=50 else (1 if x.create_role_roi<=0.03 else 2))),axis=1)
data_win_2_ios['label'] = data_win_2_ios.apply(lambda x:1 if x.create_role_num == 0 else (0 if (x.create_role_cost<=250)&(x.create_role_roi>=0.045)
                                else (1 if x.create_role_cost>=120 else (1 if x.create_role_roi<=0.02 else 2))),axis=1)
result_df = result_df.append(data_win_2[(data_win_2['label'] == 1)| (data_win_2['label'] == 0)]).append(data_win_2_ios[(data_win_2_ios['label'] == 1)|(data_win_2_ios['label'] == 0)])
temp_df = temp_df.append(data_win_2[data_win_2['label'] == 2]).append(data_win_2_ios[data_win_2_ios['label'] == 2])

In [527]:
data_win_2['label'].value_counts()

1    4169
0     699
2     222
Name: label, dtype: int64

In [528]:
data_win_2_ios['label'].value_counts()

1    1697
0     278
2     138
Name: label, dtype: int64

In [145]:
# # win = 3
# data_win_3['label'] = data_win_3.apply(lambda x:1 if x.create_role_num == 0 else 
#                                 (1 if x.create_role_cost>=100 else (1 if x.create_role_pay_sum==0 else(1 if x.create_role_pay_cost > 6000 else(1 if x.create_role_retain_1d<0.03 else(1 if x.create_role_roi<0.05 else 2))))),axis=1)
# data_win_3_ios['label'] = data_win_3_ios.apply(lambda x:1 if x.create_role_num == 0 else 
#                                 (1 if x.create_role_cost>=200 else (1 if x.create_role_pay_sum==0 else(1 if x.create_role_pay_cost > 10000 else(1 if x.create_role_retain_1d<0.03 else(1 if x.create_role_roi<0.04 else 2))))),axis=1)


# result_df = result_df.append(data_win_3[data_win_3['label'] == 1]).append(data_win_3_ios[data_win_3_ios['label'] == 1])
# temp_df = temp_df.append(data_win_3[data_win_3['label'] == 2]).append(data_win_3_ios[data_win_3_ios['label'] == 2])

In [529]:
# win = 3
data_win_3['label'] = data_win_3.apply(lambda x:1 if x.create_role_num == 0 else(1 if x.create_role_pay_num==0 else (0 if (x.create_role_cost<=70)&(x.create_role_roi>=0.07)
        else (1 if x.create_role_pay_cost > 5000 else (1 if x.create_role_retain_1d<0.05 else(1 if x.create_role_roi<0.04 else 2))))),axis=1)
data_win_3_ios['label'] = data_win_3_ios.apply(lambda x:1 if x.create_role_num == 0 else(1 if x.create_role_pay_num==0 else (0 if (x.create_role_cost<=250)&(x.create_role_roi>=0.07)
        else (1 if x.create_role_pay_cost >10000 else (1 if x.create_role_retain_1d<0.05 else(1 if x.create_role_roi<0.04 else 2))))),axis=1)
result_df = result_df.append(data_win_3[(data_win_3['label'] == 1)| (data_win_3['label'] == 0)]).append(data_win_3_ios[(data_win_3_ios['label'] == 1)|(data_win_3_ios['label'] == 0)])
temp_df = temp_df.append(data_win_3[data_win_3['label'] == 2]).append(data_win_3_ios[data_win_3_ios['label'] == 2])

In [530]:
data_win_3['label'].value_counts()

1    2489
0     556
2     426
Name: label, dtype: int64

In [531]:
data_win_3_ios['label'].value_counts()

1    1112
0     211
2     104
Name: label, dtype: int64

In [532]:
result_df.shape

(20439, 20)

In [533]:
result_df.head()

,plan_name,channel_id,source_id,data_win,source_run_datetime,create_time,plan_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_retain_1d,30_d_pay_sum,30_ROI,label
0,番茄小说-末日安卓,7162,299591,0,2020-05-01,2020-04-24,289813,1001049,1,200000.00,690.0,290.275762,3.0,66666.666667,318.0,0.00159,0.0522,4654.0,0.023270,1
7,全民投资4-末日-new-5.19_2,20953,312091,0,2020-05-19,2020-05-19,302313,1000840,1,687.12,3.0,229.040000,0.0,0.000000,0.0,0.00000,0.3333,12.0,0.017464,1
9,ba-末日-new-5.19_1,20953,312103,0,2020-05-19,2020-05-19,302325,1000840,1,552.79,3.0,184.263333,0.0,0.000000,0.0,0.00000,0.0000,0.0,0.000000,1
10,ku11-mrwc双-5.19_1,20754,312205,0,2020-05-19,2020-05-19,302427,1000840,1,596.07,3.0,198.690000,0.0,0.000000,0.0,0.00000,0.3333,0.0,0.000000,1
11,t5i-165-mrwc-5.19_7,20754,312325,0,2020-05-19,2020-05-19,302547,1000840,1,865.32,3.0,288.440000,0.0,0.000000,0.0,0.00000,0.0000,0.0,0.000000,1


In [534]:
result_df['label'].value_counts()

1    17442
0     2997
Name: label, dtype: int64

In [535]:
temp_df.shape

(3968, 20)

In [536]:
temp_df.head()

,plan_name,channel_id,source_id,data_win,source_run_datetime,create_time,plan_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_retain_1d,30_d_pay_sum,30_ROI,label
84,HSX-TT-MUS-末日518托管,20540,311483,0,2020-05-19,2020-05-18,301705,1000840,1,1635.16,33.0,49.550303,1.0,1635.16,30.0,0.018347,0.0909,42.0,0.025686,2
106,ba-末日-new-5.18_2,20953,311519,0,2020-05-19,2020-05-18,301741,1000840,1,1094.69,59.0,18.554068,0.0,0.00,0.0,0.000000,0.0169,0.0,0.000000,2
333,HSX-TT-MUS-末日-0512_7,20540,309403,0,2020-05-19,2020-05-12,299625,1000840,1,994.10,71.0,14.201429,0.0,0.00,0.0,0.000000,0.0429,648.0,0.651846,2
502,rps-142-mrwc垂-7.3_4,20754,324409,0,2020-07-11,2020-07-03,314629,1000840,1,2343.08,210.0,11.210909,0.0,0.00,0.0,0.000000,0.0383,140.0,0.059750,2
503,JIA-mgtt-末日_7.4垂直_5,6867,324613,0,2020-07-11,2020-07-04,314833,1000840,1,3515.98,213.0,16.742762,1.0,3515.98,12.0,0.003413,0.0476,1110.0,0.315701,2


## 模型识别

In [537]:
temp_df['label'] = ((temp_df['30_ROI']<=0.4)&(data['platform']==1)|(temp_df['30_ROI']<=0.3)&(data['platform']==2)).astype(int)

In [538]:
data_train = temp_df[['data_win','platform','source_run_date_amount','create_role_num','create_role_cost','create_role_pay_num','create_role_pay_cost',
                   'create_role_pay_sum','create_role_roi','create_role_retain_1d','label']]

In [539]:
data_train = temp_df[['data_win','platform','create_role_cost','create_role_pay_cost','create_role_roi','label']]

In [540]:
import lightgbm as lgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder,MinMaxScaler, StandardScaler
from sklearn.metrics import recall_score, auc, accuracy_score, f1_score, precision_score, classification_report, roc_auc_score,mean_squared_error
from sklearn.metrics import mean_absolute_error

In [541]:
# df_train, df_test = train_test_split(data_train,test_size=0.2)
target = data_train['label']
features = data_train.drop(['label'], axis=1)
X_val, x_test, Y_val, y_test = train_test_split(features,target, test_size=0.2)

In [542]:
from imblearn.over_sampling import SMOTE
smo = SMOTE(random_state=0)
X_val, Y_val = smo.fit_sample(X_val, Y_val)

In [543]:
# 使用GBDT模型预测
from sklearn.ensemble import GradientBoostingClassifier
gbdt_b = GradientBoostingClassifier()
gbdt_b.fit(X_val, Y_val)
y_predict = gbdt_b.predict_proba(x_test)

In [390]:
# # 用df_test数据集进行测试
# target_test = df_test['label']
# features_test = df_test.drop(['label'], axis=1)
# y_predict = gbdt_b.predict_proba(features_test)
# # y_predict = lgb_b_cv.predict(features_test)

In [561]:
import sklearn
sklearn.__version__

'0.23.1'

In [555]:
joblib.dump(gbdt_b,'gbdt.pkl')

['gbdt.pkl']

In [544]:
y_predict = pd.Series(y_predict[:,1])

In [552]:
y_predict_val = y_predict.apply(lambda x:1 if x>=0.5 else 0)

In [553]:
print(recall_score( y_test, y_predict_val, average=None))
print(precision_score( y_test, y_predict_val))
print(accuracy_score( y_test, y_predict_val))
print(f1_score( y_test, y_predict_val))

[0.35802469 0.78821879]
0.9153094462540716
0.7443324937027708
0.8470233609645818


In [554]:
print(classification_report( y_test, y_predict_val))

              precision    recall  f1-score   support

           0       0.16      0.36      0.22        81
           1       0.92      0.79      0.85       713

    accuracy                           0.74       794
   macro avg       0.54      0.57      0.53       794
weighted avg       0.84      0.74      0.78       794



In [432]:
y_predict_val.sum()

603

In [433]:
len(y_predict_val)

1088

In [338]:
temp_df.head()

,plan_name,channel_id,source_id,data_win,source_run_datetime,create_time,plan_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_retain_1d,30_d_pay_sum,30_ROI,label
84,HSX-TT-MUS-末日518托管,20540,311483,0,2020-05-19,2020-05-18,301705,1000840,1,1635.16,33.0,49.550303,1.0,1635.16,30.0,0.018347,0.0909,42.0,0.025686,0
85,全民投资4-末日-new-5.18_3,20953,311573,0,2020-05-19,2020-05-18,301795,1000840,1,742.19,33.0,22.490606,0.0,0.00,0.0,0.000000,0.0000,0.0,0.000000,1
89,Wu-fpftt-mrwc_1,21116,312307,0,2020-05-19,2020-05-19,302529,1000840,1,786.18,28.0,28.077857,0.0,0.00,0.0,0.000000,0.1429,0.0,0.000000,1
91,dan-末日-new-5.18_3,20953,311525,0,2020-05-19,2020-05-18,301747,1000840,1,1257.01,34.0,36.970882,0.0,0.00,0.0,0.000000,0.0294,0.0,0.000000,1
95,Wu-fpftt-mrwc_3,21116,312309,0,2020-05-19,2020-05-19,302531,1000840,1,670.75,24.0,27.947917,0.0,0.00,0.0,0.000000,0.0833,0.0,0.000000,1


In [434]:
temp_df['label'] = gbdt_b.predict(temp_df[['data_win','platform','create_role_cost','create_role_pay_cost','create_role_roi','create_role_retain_1d']])

In [435]:
temp_df['label'].value_counts()

1    4061
0    1379
Name: label, dtype: int64

In [436]:
temp_df[temp_df['label']==0].head(50)

,plan_name,channel_id,source_id,data_win,source_run_datetime,create_time,plan_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_retain_1d,30_d_pay_sum,30_ROI,label
84,HSX-TT-MUS-末日518托管,20540,311483,0,2020-05-19,2020-05-18,301705,1000840,1,1635.16,33.0,49.550303,1.0,1635.160,30.0,0.018347,0.0909,42.0,0.025686,0
89,Wu-fpftt-mrwc_1,21116,312307,0,2020-05-19,2020-05-19,302529,1000840,1,786.18,28.0,28.077857,0.0,0.000,0.0,0.000000,0.1429,0.0,0.000000,0
95,Wu-fpftt-mrwc_3,21116,312309,0,2020-05-19,2020-05-19,302531,1000840,1,670.75,24.0,27.947917,0.0,0.000,0.0,0.000000,0.0833,0.0,0.000000,0
506,jiyi-末日-7.9_1,21087,326600,0,2020-07-11,2020-07-09,316820,1000840,1,3751.57,233.0,16.170560,1.0,3751.570,60.0,0.015993,0.0905,3476.0,0.926545,0
511,ku11-mrwc-7.11_8,20754,328108,0,2020-07-11,2020-07-11,318328,1000840,1,2815.19,278.0,10.126583,0.0,0.000,0.0,0.000000,0.0612,260.0,0.092356,0
524,全民投资4-末日-new-7.8_3,20953,326171,0,2020-07-11,2020-07-08,316392,1000840,1,2403.70,99.0,24.279798,1.0,2403.700,30.0,0.012481,0.1515,30.0,0.012481,0
544,mei207-AM-5.8_8,20754,306756,0,2020-05-19,2020-05-08,296978,1001049,1,2888.26,118.0,24.476780,2.0,1444.130,54.0,0.018696,0.0763,54.0,0.018696,0
547,ba-末日-new-5.18_1,20953,311518,0,2020-05-19,2020-05-18,301740,1000840,1,3479.25,104.0,33.779126,1.0,3479.250,68.0,0.019544,0.0680,68.0,0.019544,0
549,s0103-AM-5.14_5,20754,310678,0,2020-05-19,2020-05-14,300900,1001049,1,2846.81,101.0,28.186238,0.0,0.000,0.0,0.000000,0.0594,152.0,0.053393,0
551,dan-末日-new-5.18_1,20953,311523,0,2020-05-19,2020-05-18,301745,1000840,1,2724.53,93.0,29.939890,1.0,2724.530,12.0,0.004404,0.0989,12.0,0.004404,0


In [437]:
temp_df[temp_df['label']==0]['30_d_pay_sum'].sum()/temp_df[temp_df['label']==0]['source_run_date_amount'].sum()

0.36096461391822365

In [438]:
temp_df[temp_df['label']==1]['30_d_pay_sum'].sum()/temp_df[temp_df['label']==1]['source_run_date_amount'].sum()

0.11314492685880169

# 7日开

In [451]:
data_win_7['label'] = data_win_7.apply(lambda x:1 if x.create_role_num == 0 else (0 if (x.create_role_cost<=50)&(x.create_role_roi>=0.125)&(x.create_role_pay_cost<=4000) else 1), axis=1)
data_win_7_ios['label'] = data_win_7_ios.apply(lambda x:1 if x.create_role_num == 0 else (0 if (x.create_role_cost<=100)&(x.create_role_roi>=0.11)&(x.create_role_pay_cost<=8000) else 1), axis=1)

In [452]:
data_win_7['label'].value_counts()

1    819
0    330
Name: label, dtype: int64

In [453]:
data_win_7_ios['label'].value_counts()

1    349
0    151
Name: label, dtype: int64

In [456]:
result_df = result_df.append(data_win_7).append(data_win_7_ios)

In [457]:
result = pd.concat([result_df,temp_df],axis=0)

In [458]:
result.shape

(26056, 20)

## 结果输出

In [447]:
# 没有win=3 有win=7 说明计划处于关停，则以win=7的结果为输出
# 优先级 win = 0 大于 win=3 大于 win=2 大于 win=1

# 判断过程：
# 1，是否存在win=0结果,有则以win=0为输出
# 2，没有win=0,则优先取win=3值，否则取win=2的值，否则取win=1的值
# 3，没有win=0,1,2,3,则以win=7的结果为输出

In [ ]:
# "data": {"columns": ["channel_id", "source_id", "data_win", "source_run_datetime", "create_time", "media_id", "game_id", 
#                      "platform", "source_run_date_amount", "create_role_num", "create_role_cost", "create_role_pay_num", 
#                      "create_role_pay_cost", "create_role_pay_sum", "create_role_roi", "create_role_retain_1d", "label"]

In [465]:
result['midia_id'] = 10

In [466]:
result.head()

,plan_name,channel_id,source_id,data_win,source_run_datetime,create_time,plan_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_retain_1d,30_d_pay_sum,30_ROI,label,midia_id
0,番茄小说-末日安卓,7162,299591,0,2020-05-01,2020-04-24,289813,1001049,1,200000.00,690.0,290.275762,3.0,66666.666667,318.0,0.00159,0.0522,4654.0,0.023270,1,10
7,全民投资4-末日-new-5.19_2,20953,312091,0,2020-05-19,2020-05-19,302313,1000840,1,687.12,3.0,229.040000,0.0,0.000000,0.0,0.00000,0.3333,12.0,0.017464,1,10
9,ba-末日-new-5.19_1,20953,312103,0,2020-05-19,2020-05-19,302325,1000840,1,552.79,3.0,184.263333,0.0,0.000000,0.0,0.00000,0.0000,0.0,0.000000,1,10
10,ku11-mrwc双-5.19_1,20754,312205,0,2020-05-19,2020-05-19,302427,1000840,1,596.07,3.0,198.690000,0.0,0.000000,0.0,0.00000,0.3333,0.0,0.000000,1,10
11,t5i-165-mrwc-5.19_7,20754,312325,0,2020-05-19,2020-05-19,302547,1000840,1,865.32,3.0,288.440000,0.0,0.000000,0.0,0.00000,0.0000,0.0,0.000000,1,10


In [503]:
part_1 = result[result['data_win']==0]
part_2 = result[result['data_win']==7]
temp = result.drop(part_1.index,axis=0)
temp = temp.drop(part_2.index,axis=0)

In [504]:
temp.sort_values('data_win',ascending=False,inplace=True)
temp.drop_duplicates(['channel_id','source_id','source_run_datetime'],keep='first',inplace=True)

In [505]:
temp = pd.concat([temp,part_1], axis=0)
temp.sort_values('data_win',inplace=True)
temp.drop_duplicates(['channel_id','source_id','source_run_datetime'],keep='first',inplace=True)

In [506]:
temp = pd.concat([temp,part_2], axis=0)
temp.sort_values('data_win',inplace=True)
result_out = temp.drop_duplicates(['channel_id','source_id','source_run_datetime'],keep='first')

In [507]:
result_out.shape

(15256, 21)

In [508]:
result_out.head()

,plan_name,channel_id,source_id,data_win,source_run_datetime,create_time,plan_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_retain_1d,30_d_pay_sum,30_ROI,label,midia_id
11656,t5i-165-mrwc殊-6.13_26,20754,318910,0,2020-06-14,2020-06-12,309132,1000840,1,884.06,22.0,40.184545,0.0,0.0,0.0,0.0,0.2273,0.0,0.000000,1,10
24430,0723-ft-tt-ios_6,20761,334943,0,2020-07-23,2020-07-23,325163,1000993,2,931.93,10.0,93.193000,0.0,0.0,0.0,0.0,0.1000,0.0,0.000000,1,10
24398,cxq-ttyue废土0507_4,20647,306266,0,2020-05-07,2020-05-07,296488,1000993,2,802.72,13.0,61.747692,0.0,0.0,0.0,0.0,0.1538,0.0,0.000000,1,10
24396,hus-30-废土-5.4_3,20755,305230,0,2020-05-10,2020-05-04,295452,1000993,2,875.04,11.0,79.549091,0.0,0.0,0.0,0.0,0.3636,0.0,0.000000,1,10
24393,muwo-废土-IOS-5.10_1,20954,308045,0,2020-05-10,2020-05-10,298267,1000993,2,3479.25,31.0,115.975000,0.0,0.0,0.0,0.0,0.1000,12.0,0.003449,1,10


In [511]:
result_out.shape

(15256, 21)

In [556]:
result_out[result_out['label']==0]['30_d_pay_sum'].sum()/result_out[result_out['label']==0]['source_run_date_amount'].sum()

0.7076923885895116

In [557]:
result_out[result_out['label']==1]['30_d_pay_sum'].sum()/result_out[result_out['label']==1]['source_run_date_amount'].sum()

0.12122815855854152